In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as ss
import tensorflow as tf
import math
import random
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.utils import get_custom_objects
from tensorflow.keras.layers import Activation

In [7]:
df=pd.read_excel('adj_data_new.xlsx')

In [8]:
df

,secID,revenue,COGS,TProfit,EBIT,cashCEquiv,inventories,TCA,fixedAssets,TCL,NCLWithin1Y,TShEquity,marketValue,isNew,date,year,mon
0,000002.XSHE,8.164319,5.404079,1.227682,1.412001,26.921922,84.255401,119.104076,1.231879,66.263834,12.126159,39.732353,91.040341,10,200901,2009,1
1,000002.XSHE,7.504970,4.749115,1.551611,1.657203,17.917618,103.511657,140.886939,1.343584,77.734839,12.305247,47.393511,104.454497,10,201001,2010,1
2,000002.XSHE,7.970256,4.068774,1.674455,1.823694,37.189323,150.852395,224.646644,1.210488,151.807437,18.569840,56.129244,95.548377,10,201101,2011,1
3,000002.XSHE,10.345912,6.022297,2.058143,2.187454,39.054425,214.701089,296.691048,1.579057,203.348130,15.759399,69.120707,91.040341,10,201201,2012,1
4,000002.XSHE,13.999906,8.801586,2.394886,2.694387,52.258689,289.765724,401.099312,1.578409,299.237122,30.958488,84.626427,121.074333,10,201301,2013,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7959,601919.XSHG,57.489919,55.108227,1.755805,3.910894,33.015271,1.469231,41.851645,79.573104,26.146657,8.807550,44.937511,92.150795,0,201504,2015,4
7960,601919.XSHG,71.160181,71.865800,-8.588442,-7.104659,32.512220,1.564690,45.362466,43.573505,33.500612,6.725344,37.548869,53.533278,0,201604,2016,4
7961,601919.XSHG,90.463958,82.604637,5.703036,7.226153,26.089746,2.330221,39.442195,49.114408,43.491994,8.626660,43.710580,69.164178,0,201704,2017,4
7962,601919.XSHG,120.829529,110.604883,3.845323,6.866803,33.994972,4.100906,60.304399,109.462616,88.779117,9.106630,56.352889,69.164178,0,201804,2018,4


In [9]:
ncode=np.unique(df['secID'].values)
ncode[0]

'000002.XSHE'

In [10]:
mon=[1,2,3,4]

In [11]:
year=[2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019]

In [23]:
a1=df[df['year'].isin([2010])].copy().reset_index()
a2=df[df['year'].isin([2011])].copy().reset_index()

In [36]:
k=a2[a2.columns[2:14]]-a1[a1.columns[2:14]]
k

,revenue,COGS,TProfit,EBIT,cashCEquiv,inventories,TCA,fixedAssets,TCL,NCLWithin1Y,TShEquity,marketValue
0,0.465286,-0.680342,0.122844,0.166491,19.271705,47.340738,83.759704,-0.133096,74.072598,6.264593,8.735734,-8.906120
1,3.222508,0.752982,0.303668,0.233695,21.668154,59.835195,97.209280,-0.106412,86.709725,9.038874,9.387674,18.362001
2,3.705432,2.595284,0.283782,0.485178,2.355992,69.740675,92.220547,-0.076552,85.963115,8.992044,10.217019,-12.754444
3,21.068898,13.154668,3.865130,3.824211,-3.577419,75.002036,77.125923,0.376281,71.073369,6.540139,13.246339,-8.246408
4,-0.219651,-0.121153,0.010635,0.007937,-1.127801,2.041181,0.693948,-0.017640,0.289620,0.024955,0.561809,6.871728
...,...,...,...,...,...,...,...,...,...,...,...,...
719,11.224337,9.199160,1.944730,2.119311,1.528800,3.677615,6.871310,1.700753,6.728222,-0.436004,4.104006,-36.062447
720,-0.968011,0.726030,-1.557465,-1.736247,1.197326,0.755347,2.950216,10.576177,4.217930,3.159381,8.081833,-27.890429
721,-3.965561,2.318170,-6.340063,-6.274949,1.101391,1.424912,3.342691,6.300135,11.701989,9.359364,-0.465925,-4.495161
722,-3.321196,1.716981,-4.064802,-10.946883,-3.234993,2.038263,0.181132,4.076311,17.496202,14.335163,-5.463990,-36.165611


In [37]:
k['secID']=a2['secID']
k['date']=a2['date']
k['year']=a2['year']
k['mon']=a2['mon']
k

,revenue,COGS,TProfit,EBIT,cashCEquiv,inventories,TCA,fixedAssets,TCL,NCLWithin1Y,TShEquity,marketValue,secID,date,year,mon
0,0.465286,-0.680342,0.122844,0.166491,19.271705,47.340738,83.759704,-0.133096,74.072598,6.264593,8.735734,-8.906120,000002.XSHE,201101,2011,1
1,3.222508,0.752982,0.303668,0.233695,21.668154,59.835195,97.209280,-0.106412,86.709725,9.038874,9.387674,18.362001,000002.XSHE,201102,2011,2
2,3.705432,2.595284,0.283782,0.485178,2.355992,69.740675,92.220547,-0.076552,85.963115,8.992044,10.217019,-12.754444,000002.XSHE,201103,2011,3
3,21.068898,13.154668,3.865130,3.824211,-3.577419,75.002036,77.125923,0.376281,71.073369,6.540139,13.246339,-8.246408,000002.XSHE,201104,2011,4
4,-0.219651,-0.121153,0.010635,0.007937,-1.127801,2.041181,0.693948,-0.017640,0.289620,0.024955,0.561809,6.871728,000009.XSHE,201101,2011,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
719,11.224337,9.199160,1.944730,2.119311,1.528800,3.677615,6.871310,1.700753,6.728222,-0.436004,4.104006,-36.062447,601899.XSHG,201104,2011,4
720,-0.968011,0.726030,-1.557465,-1.736247,1.197326,0.755347,2.950216,10.576177,4.217930,3.159381,8.081833,-27.890429,601919.XSHG,201101,2011,1
721,-3.965561,2.318170,-6.340063,-6.274949,1.101391,1.424912,3.342691,6.300135,11.701989,9.359364,-0.465925,-4.495161,601919.XSHG,201102,2011,2
722,-3.321196,1.716981,-4.064802,-10.946883,-3.234993,2.038263,0.181132,4.076311,17.496202,14.335163,-5.463990,-36.165611,601919.XSHG,201103,2011,3


In [38]:
newset=[]
for i in range(len(year)-1):
    temp=df[df['year'].isin([year[i]])].copy().reset_index()
    temp1=df[df['year'].isin([year[i+1]])].copy().reset_index()
    k=temp1[temp1.columns[2:14]]-temp[temp.columns[2:14]]
    k['secID']=temp1['secID']
    k['date']=temp1['date']
    k['year']=temp1['year']
    k['mon']=temp1['mon']
    newset.append(k)

In [39]:
data=pd.concat(newset)

In [41]:
data

,revenue,COGS,TProfit,EBIT,cashCEquiv,inventories,TCA,fixedAssets,TCL,NCLWithin1Y,TShEquity,marketValue,secID,date,year,mon
0,-0.659349,-0.654963,0.323929,0.245202,-9.004303,19.256256,21.782863,0.111705,11.471005,0.179088,7.661157,13.414156,000002.XSHE,201001,2010,1
1,-5.042322,-4.767555,-0.042865,-0.027840,-7.769109,27.365417,33.752739,0.170672,23.246076,2.407507,6.614867,-65.641405,000002.XSHE,201002,2010,2
2,-2.121087,-2.155288,0.088532,-0.081521,3.928839,36.443526,55.812820,0.175620,45.825817,7.423431,6.972583,-22.210325,000002.XSHE,201003,2010,3
3,1.832838,-4.441222,3.323325,3.274626,14.815009,43.248164,75.197453,-0.136395,61.592512,7.865276,9.177687,-28.477594,000002.XSHE,201004,2010,4
4,0.007698,-0.093437,0.034101,0.031045,1.515182,-0.155537,2.256662,0.021786,0.441053,0.000000,0.511912,2.819850,000009.XSHE,201001,2010,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
719,30.103732,27.931253,0.844099,1.047628,-3.864746,2.216879,-1.854292,4.480302,-3.860442,-3.938249,9.805885,10.768329,601899.XSHG,201904,2019,4
720,13.151729,10.961758,0.626122,1.450652,21.285895,1.508524,38.519982,39.432065,49.336611,6.203360,18.754982,4.556458,601919.XSHG,201901,2019,1
721,27.025957,22.123513,1.451132,3.169254,3.511391,1.115497,21.295371,42.100074,23.954271,3.169809,18.590362,11.278767,601919.XSHG,201902,2019,2
722,2.461264,1.175699,0.005055,3.237917,2.863047,-0.184107,6.539261,-3.269586,-21.854235,3.567426,9.547697,16.672960,601919.XSHG,201903,2019,3


In [42]:
data.to_excel('diff_data.xlsx')

In [46]:
df=pd.read_excel('diff_data.xlsx')

In [47]:
df

,revenue,COGS,TProfit,EBIT,cashCEquiv,inventories,TCA,fixedAssets,TCL,NCLWithin1Y,TShEquity,marketValue,secID,date,year,mon
0,-0.659349,-0.654963,0.323929,0.245202,-9.004303,19.256256,21.782863,0.111705,11.471005,0.179088,7.661157,13.414156,000002.XSHE,201001,2010,1
1,-5.042322,-4.767555,-0.042865,-0.027840,-7.769109,27.365417,33.752739,0.170672,23.246076,2.407507,6.614867,-65.641405,000002.XSHE,201002,2010,2
2,-2.121087,-2.155288,0.088532,-0.081521,3.928839,36.443526,55.812820,0.175620,45.825817,7.423431,6.972583,-22.210325,000002.XSHE,201003,2010,3
3,1.832838,-4.441222,3.323325,3.274626,14.815009,43.248164,75.197453,-0.136395,61.592512,7.865276,9.177687,-28.477594,000002.XSHE,201004,2010,4
4,0.007698,-0.093437,0.034101,0.031045,1.515182,-0.155537,2.256662,0.021786,0.441053,0.000000,0.511912,2.819850,000009.XSHE,201001,2010,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7235,30.103732,27.931253,0.844099,1.047628,-3.864746,2.216879,-1.854292,4.480302,-3.860442,-3.938249,9.805885,10.768329,601899.XSHG,201904,2019,4
7236,13.151729,10.961758,0.626122,1.450652,21.285895,1.508524,38.519982,39.432065,49.336611,6.203360,18.754982,4.556458,601919.XSHG,201901,2019,1
7237,27.025957,22.123513,1.451132,3.169254,3.511391,1.115497,21.295371,42.100074,23.954271,3.169809,18.590362,11.278767,601919.XSHG,201902,2019,2
7238,2.461264,1.175699,0.005055,3.237917,2.863047,-0.184107,6.539261,-3.269586,-21.854235,3.567426,9.547697,16.672960,601919.XSHG,201903,2019,3


In [48]:
ncode=np.unique(df['secID'].values)
ncode[0]

'000002.XSHE'

In [55]:
mon=[1,2,3,4]

In [56]:
trainyear=[2010,2011,2012,2013]
trainyear1=[2011,2012,2013,2014]
trainyear2=[2012,2013,2014,2015]

In [57]:
train=[]
for i in range(len(ncode)):
    temp=df[df['secID'].isin([ncode[i]])].copy()
    temp1=temp[temp['year'].isin(trainyear)].copy()
    for j in mon:
        temp2=temp1[temp1['mon'].isin([j])].copy()
        values=temp2.loc[:,'revenue':'marketValue'].copy()
        val=values.values
        train.append(val)

In [58]:

for i in range(len(ncode)):
    temp=df[df['secID'].isin([ncode[i]])].copy()
    temp1=temp[temp['year'].isin(trainyear1)].copy()
    for j in mon:
        temp2=temp1[temp1['mon'].isin([j])].copy()
        values=temp2.loc[:,'revenue':'marketValue'].copy()
        val=values.values
        train.append(val)

In [59]:

for i in range(len(ncode)):
    temp=df[df['secID'].isin([ncode[i]])].copy()
    temp1=temp[temp['year'].isin(trainyear2)].copy()
    for j in mon:
        temp2=temp1[temp1['mon'].isin([j])].copy()
        values=temp2.loc[:,'revenue':'marketValue'].copy()
        val=values.values
        train.append(val)

In [60]:
x_train=np.array(train)

In [61]:
x_train.shape

(2172, 4, 12)

In [62]:
testyear=[2014]
testyear1=[2015]
testyear2=[2016]

In [63]:
test=[]
for i in range(len(ncode)):
    temp=df[df['secID'].isin([ncode[i]])].copy()
    temp1=temp[temp['year'].isin(testyear)].copy()
    for j in mon:
        temp2=temp1[temp1['mon'].isin([j])].copy()
        values=temp2.loc[:,'revenue':'marketValue'].copy()
        
        val=values.values
        test.append(val)

In [64]:
for i in range(len(ncode)):
    temp=df[df['secID'].isin([ncode[i]])].copy()
    temp1=temp[temp['year'].isin(testyear1)].copy()
    for j in mon:
        temp2=temp1[temp1['mon'].isin([j])].copy()
        values=temp2.loc[:,'revenue':'marketValue'].copy()
        
        val=values.values
        test.append(val)

In [65]:
for i in range(len(ncode)):
    temp=df[df['secID'].isin([ncode[i]])].copy()
    temp1=temp[temp['year'].isin(testyear2)].copy()
    for j in mon:
        temp2=temp1[temp1['mon'].isin([j])].copy()
        values=temp2.loc[:,'revenue':'marketValue'].copy()
        
        val=values.values
        test.append(val)

In [66]:
y_train=np.array(test)

In [67]:
y_train=y_train.reshape(2172,12)
y_train.shape

(2172, 12)

In [68]:
RNN_CELLSIZE = 100
SEQLEN = x_train.shape[1]
VAR=x_train.shape[2]
BATCHSIZE=256
LR = 0.005

The Maximum Likelihood Function is $\theta ^{MLE}=max\prod_{i=1}^{n}\frac{1}{\sqrt{2\pi g_{\theta }(\mathbf{x_{i}})^{2} }}exp(\frac{-(y_{i}-f_{\theta }(\mathbf{x_{i}})^{2})}{2g_{\theta }(\mathbf{x_{i}})^{2}})$. And the loss function of the neural network is $\theta ^{MLE}=min\sum_{i=1}^{n} (log( g_{\theta }(\mathbf{x_{i}}))+\frac{(y_{i}-f_{\theta }(\mathbf{x_{i}}))^{2}}{2g_{\theta }(\mathbf{x_{i}})^{2}})$

In [73]:
class GRUModel(tf.keras.Model):
    def __init__(self, seq_length, cell_size,batchsize,var):
        super().__init__()
        self.batchsize=batchsize
        self.var=var
        self.seq_length = seq_length
        self.cell_size = cell_size
    
        self.layer1 = tf.keras.layers.Reshape((self.seq_length,self.var), batch_size = self.batchsize)
        #self.layer2=  tf.keras.layers.Dense(10,input_shape=(self.seq_length,self.var),activation='relu',
                                          #kernel_initializer=tf.keras.initializers.TruncatedNormal(mean=1,stddev=1),
                                           #bias_initializer=tf.keras.initializers.Constant(1))
        self.layer_GRU = tf.keras.layers.GRU(self.cell_size,activation='relu', return_sequences=False)
        self.layer_dense = tf.keras.layers.Dense(12,activation='softplus',bias_initializer=tf.keras.initializers.Constant(1),
                                                kernel_initializer=tf.keras.initializers.TruncatedNormal(mean=2,stddev=1))
        self.layer_dense1 = tf.keras.layers.Dense(12,activation='linear')
  
    def call(self, inputs):
        x1 = self.layer1(inputs)
        #x1 = self.layer2(x1)
        x2 = self.layer_GRU(x1)
        output1=self.layer_dense(x2)
        output2=self.layer_dense1(x2)
        
        return output1,output2

In [74]:
def loss_fun(y_pred1,y_pred2,y_test):
    #loss=tf.math.log(y_pred1)+tf.clip_by_value((tf.square(y_test-y_pred2))/(tf.square(y_pred1)*2),0,100)
    loss=tf.math.log(y_pred1)+tf.square(y_test-y_pred2)/(tf.square(y_pred1)*2)
    #loss=tf.math.log(tf.reduce_mean(y_pred1,axis=0))+tf.reduce_mean(tf.square(y_test-y_pred2),axis=0)/(tf.square(tf.reduce_mean(y_pred1,axis=0))*2)
    return tf.square(tf.reduce_mean(loss))

In [75]:
model = GRUModel(SEQLEN,RNN_CELLSIZE,BATCHSIZE, VAR)
optimizer = tf.keras.optimizers.Adam(learning_rate = LR)

In [76]:
for epoch in range(100):
    with tf.GradientTape() as tape:
        y_pred = model(x_train)

        loss=loss_fun(y_pred[0],y_pred[1],y_train)
        #loss=tf.reduce_mean((y_pred-y_train)**2)
        
        if epoch%1 == 0:
            print("epoch: {}, loss: {}".format(epoch, loss))
  
    grads = tape.gradient(loss, model.variables)
    #print(grads)
    optimizer.apply_gradients(zip(grads,model.variables))


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

epoch: 0, loss: 17.276630401611328
epoch: 1, loss: 15.35042667388916
epoch: 2, loss: 13.691048622131348
epoch: 3, loss: 12.22148323059082
epoch: 4, loss: 10.913783073425293
epoch: 5, loss: 9.768458366394043
epoch: 6, loss: 8.799027442932129
epoch: 7, loss: 8.041680335998535
epoch: 8, loss: 7.506962299346924
epoch: 9, loss: 7.108467102050781
epoch: 10, loss: 6.715259552001953
epoch: 11, loss: 6.292076587677002
epoch: 12, loss: 5.891685485839844
epoch: 13, loss: 5.569540977478027
epoch: 14, loss: 5.333621

In [124]:
ytestyear1=[2015,2016,2017,2018]

In [125]:
test_x=[]
for i in range(len(ncode)):
    temp=df[df['secID'].isin([ncode[i]])].copy()
    temp1=temp[temp['year'].isin(ytestyear1)].copy()
    for j in mon:
        temp2=temp1[temp1['mon'].isin([j])].copy()
        values=temp2.loc[:,'revenue':'marketValue'].copy()

        val=values.values
        test_x.append(val)

In [126]:
x_test=np.array(test_x)
x_test.shape

(724, 4, 12)

In [127]:
result=model.predict(x_test)

In [128]:
EBIT_mkt=result[1][:,3]

risk-adjusted EBIT

In [129]:
adje=EBIT_mkt/result[0][:,3]

In [130]:
codelist=[]
for i in range(len(ncode)):
    code=ncode[i]
    for i in range(4):
        codelist.append(code)
codelist    

['000002.XSHE',
 '000002.XSHE',
 '000002.XSHE',
 '000002.XSHE',
 '000009.XSHE',
 '000009.XSHE',
 '000009.XSHE',
 '000009.XSHE',
 '000012.XSHE',
 '000012.XSHE',
 '000012.XSHE',
 '000012.XSHE',
 '000027.XSHE',
 '000027.XSHE',
 '000027.XSHE',
 '000027.XSHE',
 '000031.XSHE',
 '000031.XSHE',
 '000031.XSHE',
 '000031.XSHE',
 '000039.XSHE',
 '000039.XSHE',
 '000039.XSHE',
 '000039.XSHE',
 '000043.XSHE',
 '000043.XSHE',
 '000043.XSHE',
 '000043.XSHE',
 '000046.XSHE',
 '000046.XSHE',
 '000046.XSHE',
 '000046.XSHE',
 '000060.XSHE',
 '000060.XSHE',
 '000060.XSHE',
 '000060.XSHE',
 '000061.XSHE',
 '000061.XSHE',
 '000061.XSHE',
 '000061.XSHE',
 '000063.XSHE',
 '000063.XSHE',
 '000063.XSHE',
 '000063.XSHE',
 '000078.XSHE',
 '000078.XSHE',
 '000078.XSHE',
 '000078.XSHE',
 '000090.XSHE',
 '000090.XSHE',
 '000090.XSHE',
 '000090.XSHE',
 '000100.XSHE',
 '000100.XSHE',
 '000100.XSHE',
 '000100.XSHE',
 '000157.XSHE',
 '000157.XSHE',
 '000157.XSHE',
 '000157.XSHE',
 '000158.XSHE',
 '000158.XSHE',
 '000158

In [131]:
monlist=mon*len(ncode)

In [132]:
da=pd.DataFrame()

In [133]:
da['code']=codelist
da['adjEBIT']=adje
da['fo_EBIT']=result[1][:,3]
da['mon']=monlist
da['year']=2019

In [134]:
da

,code,adjEBIT,fo_EBIT,mon,year
0,000002.XSHE,0.175085,7.019540,1,2019
1,000002.XSHE,0.155225,4.340287,2,2019
2,000002.XSHE,0.159326,8.948182,3,2019
3,000002.XSHE,0.079160,4.073549,4,2019
4,000009.XSHE,0.021815,0.028398,1,2019
...,...,...,...,...,...
719,601899.XSHG,0.152115,0.632424,4,2019
720,601919.XSHG,0.026835,0.530319,1,2019
721,601919.XSHG,0.059931,0.811415,2,2019
722,601919.XSHG,0.103236,1.798780,3,2019


In [135]:
mul=(len(da['year']))/4
mul

181.0

In [136]:
date=['2018-12-28','2019-03-29','2019-06-28','2019-09-30']*181
date

['2018-12-28',
 '2019-03-29',
 '2019-06-28',
 '2019-09-30',
 '2018-12-28',
 '2019-03-29',
 '2019-06-28',
 '2019-09-30',
 '2018-12-28',
 '2019-03-29',
 '2019-06-28',
 '2019-09-30',
 '2018-12-28',
 '2019-03-29',
 '2019-06-28',
 '2019-09-30',
 '2018-12-28',
 '2019-03-29',
 '2019-06-28',
 '2019-09-30',
 '2018-12-28',
 '2019-03-29',
 '2019-06-28',
 '2019-09-30',
 '2018-12-28',
 '2019-03-29',
 '2019-06-28',
 '2019-09-30',
 '2018-12-28',
 '2019-03-29',
 '2019-06-28',
 '2019-09-30',
 '2018-12-28',
 '2019-03-29',
 '2019-06-28',
 '2019-09-30',
 '2018-12-28',
 '2019-03-29',
 '2019-06-28',
 '2019-09-30',
 '2018-12-28',
 '2019-03-29',
 '2019-06-28',
 '2019-09-30',
 '2018-12-28',
 '2019-03-29',
 '2019-06-28',
 '2019-09-30',
 '2018-12-28',
 '2019-03-29',
 '2019-06-28',
 '2019-09-30',
 '2018-12-28',
 '2019-03-29',
 '2019-06-28',
 '2019-09-30',
 '2018-12-28',
 '2019-03-29',
 '2019-06-28',
 '2019-09-30',
 '2018-12-28',
 '2019-03-29',
 '2019-06-28',
 '2019-09-30',
 '2018-12-28',
 '2019-03-29',
 '2019-06-

In [137]:
da['date']=date
da

,code,adjEBIT,fo_EBIT,mon,year,date
0,000002.XSHE,0.175085,7.019540,1,2019,2018-12-28
1,000002.XSHE,0.155225,4.340287,2,2019,2019-03-29
2,000002.XSHE,0.159326,8.948182,3,2019,2019-06-28
3,000002.XSHE,0.079160,4.073549,4,2019,2019-09-30
4,000009.XSHE,0.021815,0.028398,1,2019,2018-12-28
...,...,...,...,...,...,...
719,601899.XSHG,0.152115,0.632424,4,2019,2019-09-30
720,601919.XSHG,0.026835,0.530319,1,2019,2018-12-28
721,601919.XSHG,0.059931,0.811415,2,2019,2019-03-29
722,601919.XSHG,0.103236,1.798780,3,2019,2019-06-28


In [138]:
da.to_excel('2019diffdf.xlsx')